In [1]:
#Imports Libs
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
#Install folium
!pip install folium
import folium 

In [3]:
#Download the json data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [11]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [12]:
CLIENT_ID = 'QLQESY4MGLW1DXYO5YZRIKJ4SDSLWKHSXRMGK2BEW4GWYCMW' 
CLIENT_SECRET = 'J4QCHAYC54BV5TDYZP5YZRPJ3T5EUEDLFI2HMJ1FIZG3TMFQ' 
VERSION = '20180605' 

In [13]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [14]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] 
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] 

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [15]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    40.87655077879964, 
    -73.91065965862981, 
    500, 
    100)

In [16]:
results = requests.get(url).json()

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
LIMIT = 39
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [22]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.000000,0.015873,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0,0.0,0.000000
1,Carnegie Hill,0.0,0.0,0.0,0.000000,0.011364,0.0,0.0,0.0,0.011364,...,0.0,0.022727,0.0,0.0,0.0,0.011364,0.045455,0.0,0.0,0.034091
2,Central Harlem,0.0,0.0,0.0,0.065217,0.043478,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
3,Chelsea,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.010000,0.020000,0.0,0.0,0.000000
4,Chinatown,0.0,0.0,0.0,0.000000,0.030000,0.0,0.0,0.0,0.000000,...,0.0,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.010000


In [24]:
#manhattan_grouped['Movie Theater'].isin([0])
manhattan_grouped.sort_values(by=['Movie Theater'],ascending=True)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
19,Lower East Side,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000
21,Manhattanville,0.000000,0.00,0.00,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22,Marble Hill,0.000000,0.00,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000
23,Midtown,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24,Midtown South,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000
25,Morningside Heights,0.000000,0.00,0.00,0.000000,0.073171,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26,Murray Hill,0.000000,0.00,0.00,0.000000,0.031579,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010526,0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.000000,0.010526
27,Noho,0.000000,0.00,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.030000,0.020000,0.000000,0.000000,0.020000
20,Manhattan Valley,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.023256,0.000000,0.000000,0.000000,0.000000,0.023256,0.023256,0.000000,0.046512
28,Roosevelt Island,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.13
1    Coffee Shop  0.06
2          Hotel  0.06
3            Gym  0.05
4  Memorial Site  0.05


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.05
2  Italian Restaurant  0.05
3           Wine Shop  0.05
4         Yoga Studio  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1    Seafood Restaurant  0.04
2  Gym / Fitness Center  0.04
3    Chinese Restaurant  0.04
4   Fried Chicken Joint  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.08
1          Art Gallery  0.07
2       Ice Cream Shop  0.04
3  American Restaurant  0.04
4                 Café  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.08
1              Bakery  0.06
2        Dessert Shop  0.04
3        Optical Shop  0.03
4      Ice Cream Shop  0.03


----Civic Center----
                  venue  freq
0     

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = len(manhattan_grouped.columns)-1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,319th Most Common Venue,320th Most Common Venue,321th Most Common Venue,322th Most Common Venue,323th Most Common Venue,324th Most Common Venue,325th Most Common Venue,326th Most Common Venue,327th Most Common Venue,328th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Plaza,Gourmet Shop,Burger Joint,Food Court,...,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Accessories Store
1,Carnegie Hill,Coffee Shop,Wine Shop,Café,Italian Restaurant,Yoga Studio,Gym,Gym / Fitness Center,Bookstore,Japanese Restaurant,...,Library,Lebanese Restaurant,Laundry Service,Latin American Restaurant,Tailor Shop,Korean Restaurant,Kitchen Supply Store,Kids Store,Kebab Restaurant,Accessories Store
2,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,Fried Chicken Joint,Bar,French Restaurant,Seafood Restaurant,Bookstore,...,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Accessories Store
3,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,American Restaurant,Café,Italian Restaurant,Bakery,French Restaurant,Market,...,Men's Store,Miscellaneous Shop,Music School,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Accessories Store
4,Chinatown,Chinese Restaurant,Bakery,Dessert Shop,Bubble Tea Shop,Ice Cream Shop,Spa,Bar,Cocktail Bar,Hotpot Restaurant,...,Miscellaneous Shop,Modern European Restaurant,Newsstand,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Music School,Nail Salon,Accessories Store


In [37]:
movie_theater = neighborhoods_venues_sorted[columns]
movie_theater.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,319th Most Common Venue,320th Most Common Venue,321th Most Common Venue,322th Most Common Venue,323th Most Common Venue,324th Most Common Venue,325th Most Common Venue,326th Most Common Venue,327th Most Common Venue,328th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Plaza,Gourmet Shop,Burger Joint,Food Court,...,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Accessories Store
1,Carnegie Hill,Coffee Shop,Wine Shop,Café,Italian Restaurant,Yoga Studio,Gym,Gym / Fitness Center,Bookstore,Japanese Restaurant,...,Library,Lebanese Restaurant,Laundry Service,Latin American Restaurant,Tailor Shop,Korean Restaurant,Kitchen Supply Store,Kids Store,Kebab Restaurant,Accessories Store
2,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,Fried Chicken Joint,Bar,French Restaurant,Seafood Restaurant,Bookstore,...,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Accessories Store
3,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,American Restaurant,Café,Italian Restaurant,Bakery,French Restaurant,Market,...,Men's Store,Miscellaneous Shop,Music School,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Accessories Store
4,Chinatown,Chinese Restaurant,Bakery,Dessert Shop,Bubble Tea Shop,Ice Cream Shop,Spa,Bar,Cocktail Bar,Hotpot Restaurant,...,Miscellaneous Shop,Modern European Restaurant,Newsstand,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Music School,Nail Salon,Accessories Store


In [38]:
movie_theater_sorted = movie_theater[movie_theater.isin(['Movie Theater'])]

In [39]:
movie_theater_head = neighborhoods_venues_sorted['Neighborhood']
df_movie_theater_sorted = pd.merge(movie_theater_head,movie_theater_sorted,left_index=True,right_index=True )
df_movie_theater_sorted.head()

,Neighborhood_x,Neighborhood_y,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,319th Most Common Venue,320th Most Common Venue,321th Most Common Venue,322th Most Common Venue,323th Most Common Venue,324th Most Common Venue,325th Most Common Venue,326th Most Common Venue,327th Most Common Venue,328th Most Common Venue
0,Battery Park City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carnegie Hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Central Harlem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chinatown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Movie Theater,NaN,NaN,NaN


In [40]:
df_data = df_movie_theater_sorted.dropna(axis=1,how='all')
df_data

,Neighborhood_x,27th Most Common Venue,29th Most Common Venue,34th Most Common Venue,43th Most Common Venue,56th Most Common Venue,225th Most Common Venue,261th Most Common Venue,270th Most Common Venue,295th Most Common Venue,...,313th Most Common Venue,314th Most Common Venue,316th Most Common Venue,318th Most Common Venue,319th Most Common Venue,320th Most Common Venue,321th Most Common Venue,322th Most Common Venue,325th Most Common Venue,326th Most Common Venue
0,Battery Park City,Movie Theater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carnegie Hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Movie Theater,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Central Harlem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Movie Theater,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chelsea,NaN,NaN,NaN,NaN,Movie Theater,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chinatown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Movie Theater,NaN
5,Civic Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Movie Theater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Clinton,NaN,NaN,Movie Theater,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,East Harlem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Movie Theater,NaN,NaN,NaN,NaN,NaN
8,East Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Movie Theater,NaN,NaN
9,Financial District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Movie Theater,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
data_columns = ['Neighborhood','Rank']
tempdata = pd.DataFrame(columns=data_columns)
data_rank = pd.DataFrame(columns=data_columns)

for i in range(0,len(df_data.columns)):
    rank = df_data.iloc[i].dropna()
    rank_data = pd.DataFrame(rank)
    new=pd.DataFrame({'Neighborhood': rank_data.iloc[0],
                      'Rank':rank_data.index[1]})
    data_rank=data_rank.append(new,ignore_index=True)
data_rank

,Neighborhood,Rank
0,Battery Park City,27th Most Common Venue
1,Carnegie Hill,270th Most Common Venue
2,Central Harlem,316th Most Common Venue
3,Chelsea,56th Most Common Venue
4,Chinatown,325th Most Common Venue
5,Civic Center,313th Most Common Venue
6,Clinton,34th Most Common Venue
7,East Harlem,319th Most Common Venue
8,East Village,322th Most Common Venue
9,Financial District,318th Most Common Venue


In [42]:
#data_rank_s = data_rank.sort_values(by='Rank',ascending = False).reset_index(drop= True).map(lambda x: str(x)[:-1])
data_rank_s = data_rank
data_rank_s['Rank'] =  data_rank_s['Rank'].map(lambda x: str(x)[:-20])

In [43]:
data_rank_sorted = data_rank_s.sort_values(by=['Rank'],ascending = True).reset_index(drop= True)
data_rank_sorted

,Neighborhood,Rank
0,Greenwich Village,225
1,Lenox Hill,261
2,Battery Park City,27
3,Carnegie Hill,270
4,Manhattan Valley,295
5,Civic Center,313
6,Manhattanville,314
7,Central Harlem,316
8,Lower East Side,316
9,Financial District,318


In [44]:
manhattan_merged = manhattan_data


manhattan_merged = manhattan_merged.join(data_rank_sorted.set_index('Neighborhood'), on='Neighborhood').dropna(axis =0,how = 'any')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Rank
1,Manhattan,Chinatown,40.715618,-73.994279,325
3,Manhattan,Inwood,40.867684,-73.921210,318
4,Manhattan,Hamilton Heights,40.823604,-73.949688,321
5,Manhattan,Manhattanville,40.816934,-73.957385,314
6,Manhattan,Central Harlem,40.815976,-73.943211,316


In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

num = 1

x = np.arange(num)
ys = [i + x + (i*x)**2 for i in range(num)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood']):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters